### MÓDULO: Sistemas de Aprendizaje Automático   

# Predicción del consumo de energía eléctrica con Redes LSTM y GRU: Un estudio comparativo.

## Justificación:

La predicción del consumo de energía eléctrica es crucial para la gestión eficiente de los recursos energéticos. En este ejercicio, aplicarás técnicas de aprendizaje profundo para predecir el consumo de energía eléctrica utilizando un conjunto de datos de series temporales.

## Tarea a realizar:

- Carga y explora el conjunto de datos ["Individual household electric power consumption dataset"](https://archive.ics.uci.edu/dataset/235/individual+household+electric+power+consumption) (disponible en UCI Machine Learning Repository). Este conjunto de datos contiene mediciones del consumo de energía eléctrica en un hogar durante un período de cuatro años.
- Realiza un preprocesamiento adecuado de los datos, incluyendo la limpieza de valores faltantes, la normalización y la preparación de los datos para su uso en modelos de series temporales.
- Implementa un modelo de Red LSTM para predecir el consumo de energía eléctrica.
- Implementa un modelo de Red GRU para realizar la misma tarea.
- Evalúa el rendimiento de ambos modelos utilizando métricas como el Error Cuadrático Medio (RMSE) y el Error Absoluto Medio (MAE).
- Compara los resultados obtenidos por los modelos LSTM y GRU, analizando sus fortalezas y debilidades en este contexto específico.

En definitiva, se debe elaborar un informe conciso, usando un cuaderno Jupyter, que resuma los hallazgos, incluyendo gráficos que visualicen las predicciones de ambos modelos y una tabla comparativa de las métricas de evaluación.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from keras.models import Sequential
from keras.layers import LSTM, GRU, Dense

# 1. Carga y exploración inicial
# Ajusta la ruta al archivo CSV del dataset.
df = pd.read_csv("household_power_consumption.txt", sep=";", 
                 parse_dates={"datetime": ["Date", "Time"]}, 
                 infer_datetime_format=True, low_memory=False, na_values=["?"])

# 2. Limpieza y normalización (ejemplo)
df = df.dropna()
scaler = MinMaxScaler()
df[["Global_active_power"]] = scaler.fit_transform(df[["Global_active_power"]])

# 3. Preparación de datos de serie temporal (ejemplo)
# Selecciona cuántos pasos se usarán para predecir
timesteps = 24
def create_sequences(data, ts=timesteps):
    X, y = [], []
    for i in range(len(data) - ts):
        X.append(data[i : i + ts])
        y.append(data[i + ts])
    return np.array(X), np.array(y)

series_data = df["Global_active_power"].values
X_all, y_all = create_sequences(series_data)

# 4. División en entrenamiento y prueba
train_size = int(len(X_all)*0.8)
X_train, X_test = X_all[:train_size], X_all[train_size:]
y_train, y_test = y_all[:train_size], y_all[train_size:]

# 5. Modelo LSTM
model_lstm = Sequential()
model_lstm.add(LSTM(64, input_shape=(timesteps, 1)))
model_lstm.add(Dense(1))
model_lstm.compile(optimizer="adam", loss="mse")
model_lstm.fit(X_train.reshape(-1,timesteps,1), y_train, epochs=5, batch_size=32, verbose=1)

# 6. Modelo GRU
model_gru = Sequential()
model_gru.add(GRU(64, input_shape=(timesteps, 1)))
model_gru.add(Dense(1))
model_gru.compile(optimizer="adam", loss="mse")
model_gru.fit(X_train.reshape(-1,timesteps,1), y_train, epochs=5, batch_size=32, verbose=1)

# 7. Evaluación
pred_lstm = model_lstm.predict(X_test.reshape(-1,timesteps,1))
pred_gru = model_gru.predict(X_test.reshape(-1,timesteps,1))

rmse_lstm = mean_squared_error(y_test, pred_lstm, squared=False)
mae_lstm = mean_absolute_error(y_test, pred_lstm)
rmse_gru = mean_squared_error(y_test, pred_gru, squared=False)
mae_gru = mean_absolute_error(y_test, pred_gru)

print("LSTM -> RMSE:", rmse_lstm, "MAE:", mae_lstm)
print("GRU  -> RMSE:", rmse_gru,  "MAE:", mae_gru)

# 8. Visualización (ejemplo comparativo)
plt.figure(figsize=(10,5))
plt.plot(y_test[:100], label="Real")
plt.plot(pred_lstm[:100], label="Pred LSTM")
plt.plot(pred_gru[:100], label="Pred GRU")
plt.legend()
plt.show()


/tmp/ipykernel_69978/151398205.py:12: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df = pd.read_csv("household_power_consumption.txt", sep=";",
/tmp/ipykernel_69978/151398205.py:12: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv("household_power_consumption.txt", sep=";",
/tmp/ipykernel_69978/151398205.py:12: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df = pd.read_csv("household_power_consumption.txt", sep=";",
2025-03-20 21:31:49.928688: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] faile

Epoch 1/5
51232/51232 ━━━━━━━━━━━━━━━━━━━━ 318s 6ms/step - loss: 6.7529e-04
Epoch 2/5
51232/51232 ━━━━━━━━━━━━━━━━━━━━ 322s 6ms/step - loss: 5.9136e-04
Epoch 3/5
 9697/51232 ━━━━━━━━━━━━━━━━━━━━ 4:25 6ms/step - loss: 5.8991e-04

KeyboardInterrupt: 